In [37]:
import pandas as pd

**Third country nationals found to be illegally present - annual data (rounded)**

---
Units of measure are absolute numbers of persons.

In [38]:
df = pd.read_csv('../../data/illegally_present_third_country_nationals.csv')

In [39]:
df.head()

,DATAFLOW,LAST UPDATE,freq,reason,apprehen,citizen,sex,age,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Total,Andorra,Total,Total,Person,Austria,2021,0.0,NaN,NaN
1,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Total,Andorra,Total,Total,Person,Austria,2022,0.0,NaN,NaN
2,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Total,Andorra,Total,Total,Person,Belgium,2021,0.0,NaN,NaN
3,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Total,Andorra,Total,Total,Person,Belgium,2022,0.0,NaN,NaN
4,ESTAT:MIGR_EIPRE(1.0),04/11/24 23:00:00,Annual,Illegal entry,Total,Andorra,Total,Total,Person,Belgium,2023,0.0,NaN,NaN


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49510 entries, 0 to 49509
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     49510 non-null  object 
 1   LAST UPDATE  49510 non-null  object 
 2   freq         49510 non-null  object 
 3   reason       49510 non-null  object 
 4   apprehen     49510 non-null  object 
 5   citizen      49510 non-null  object 
 6   sex          49510 non-null  object 
 7   age          49510 non-null  object 
 8   unit         49510 non-null  object 
 9   geo          49510 non-null  object 
 10  TIME_PERIOD  49510 non-null  int64  
 11  OBS_VALUE    49300 non-null  float64
 12  OBS_FLAG     306 non-null    object 
 13  CONF_STATUS  0 non-null      float64
dtypes: float64(2), int64(1), object(11)
memory usage: 5.3+ MB


In [41]:
df.duplicated().sum()

np.int64(0)

In [42]:
df.isnull().sum()

DATAFLOW           0
LAST UPDATE        0
freq               0
reason             0
apprehen           0
citizen            0
sex                0
age                0
unit               0
geo                0
TIME_PERIOD        0
OBS_VALUE        210
OBS_FLAG       49204
CONF_STATUS    49510
dtype: int64

In [43]:
df = df.drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "apprehen", "unit", "OBS_FLAG", "CONF_STATUS"])

In [44]:
df = df[df['OBS_VALUE'] != 0]
df = df[df['geo'] != 'European Union - 27 countries (from 2020)']
df = df[df['geo'] != 'European Union - 28 countries (2013-2020)']

In [45]:
for col in df:
    unique_vals = df[col].unique()
    print(f"{col}: {unique_vals[:40]}")
    if len(unique_vals) > 40:
        print(f"... and {len(unique_vals) - 40} others")

reason: ['Illegal entry' 'Overstay' 'Other reason']
citizen: ['United Arab Emirates' 'Afghanistan' 'Albania' 'Armenia' 'Angola'
 'Argentina' 'Asia' 'Australia' 'Azerbaijan' 'Bosnia and Herzegovina'
 'Bangladesh' 'Burkina Faso' 'Bahrain' 'Burundi' 'Benin' 'Bolivia'
 'Brazil' 'Bahamas' 'Bhutan' 'Botswana' 'Belarus' 'Belize' 'Canada'
 'Democratic Republic of the Congo' 'Central African Republic' 'Congo'
 'Côte d’Ivoire' 'Chile' 'Cameroon' 'China' 'Colombia' 'Costa Rica' 'Cuba'
 'Cabo Verde' 'Djibouti' 'Dominica' 'Dominican Republic' 'Algeria'
 'Ecuador' 'Egypt']
... and 118 others
sex: ['Total']
age: ['Total']
geo: ['Switzerland' 'Germany' 'Hungary' 'Bulgaria' 'Cyprus' 'Czechia' 'Denmark'
 'Estonia' 'Greece' 'Finland' 'France' 'Croatia' 'Italy' 'Liechtenstein'
 'Latvia' 'Netherlands' 'Norway' 'Poland' 'Romania' 'Slovenia' 'Slovakia'
 'Belgium' 'Luxembourg' 'Montenegro' 'Ireland' 'Malta' 'Austria' 'Spain'
 'Iceland' 'Lithuania' 'Portugal' 'Sweden']
TIME_PERIOD: [2022 2023 2021 2020 2019]
O

In [46]:
# remove all columns with only one unique value
df = df.loc[:, df.nunique() > 1]

In [ ]:
df = df.rename(columns={'OBS_VALUE': 'Illegal Third Country Nationals', 'TIME_PERIOD': 'year'})

In [47]:
df

,reason,citizen,geo,TIME_PERIOD,OBS_VALUE
103,Illegal entry,United Arab Emirates,Switzerland,2022,5.0
104,Illegal entry,United Arab Emirates,Switzerland,2023,5.0
112,Illegal entry,United Arab Emirates,Germany,2022,15.0
113,Illegal entry,United Arab Emirates,Germany,2023,30.0
135,Illegal entry,United Arab Emirates,Hungary,2022,10.0
...,...,...,...,...,...
49452,Other reason,Zimbabwe,France,2023,5.0
49491,Other reason,Zimbabwe,Norway,2023,5.0
49492,Other reason,Zimbabwe,Poland,2021,15.0
49493,Other reason,Zimbabwe,Poland,2022,5.0


In [48]:
nan_df = df[df.isna().any(axis=1)]
nan_df

grouped = nan_df.groupby(['citizen', 'geo']).size().reset_index(name='count')

table = grouped.pivot(index='citizen', columns='geo', values='count').fillna(0)

print("Count of NaN Observations by Citizen and Geo:")
table
# Iterate through the pivot table and print only non-zero counts
for citizen, row in table.iterrows():
    for geo, count in row.items():
        if count != 0:
            print(f"Citizen: {citizen}, Geo: {geo} -> Count: {int(count)}")


Count of NaN Observations by Citizen and Geo:
Citizen: Croatia, Geo: Austria -> Count: 3
Citizen: Croatia, Geo: Belgium -> Count: 6
Citizen: Croatia, Geo: Bulgaria -> Count: 6
Citizen: Croatia, Geo: Croatia -> Count: 6
Citizen: Croatia, Geo: Cyprus -> Count: 6
Citizen: Croatia, Geo: Czechia -> Count: 6
Citizen: Croatia, Geo: Denmark -> Count: 3
Citizen: Croatia, Geo: Estonia -> Count: 6
Citizen: Croatia, Geo: Finland -> Count: 6
Citizen: Croatia, Geo: Germany -> Count: 6
Citizen: Croatia, Geo: Greece -> Count: 6
Citizen: Croatia, Geo: Hungary -> Count: 6
Citizen: Croatia, Geo: Iceland -> Count: 3
Citizen: Croatia, Geo: Ireland -> Count: 15
Citizen: Croatia, Geo: Italy -> Count: 9
Citizen: Croatia, Geo: Latvia -> Count: 6
Citizen: Croatia, Geo: Liechtenstein -> Count: 6
Citizen: Croatia, Geo: Lithuania -> Count: 9
Citizen: Croatia, Geo: Luxembourg -> Count: 6
Citizen: Croatia, Geo: Malta -> Count: 9
Citizen: Croatia, Geo: Montenegro -> Count: 6
Citizen: Croatia, Geo: Netherlands -> Coun

In [49]:
df.to_csv("../../processed_data/illegally_present_third_country_nationals.csv", index=False)